 data processing 

In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


In [3]:
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [5]:

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
application_df.shape

(34299, 12)

In [7]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis=1)

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# All application types with less than 500 applications 
application_types_to_replace = ['T17', 'T29', 'T15', 'T25', 'T14', 'T2', 'T12', 'T13', 'T9']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# All classifications less than 100
val_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = val_counts.loc[val_counts < 100].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Convert categorical data to numeric with `pd.get_dummies`
app_dummies = pd.get_dummies(application_df)

# Split our preprocessed data into our features and target arrays
target = app_dummies['IS_SUCCESSFUL']
app_dummies = app_dummies.drop('IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
(X_train, X_test, y_train, y_test) = train_test_split(app_dummies, target, random_state=76)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

 Adjusting model structure

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=49))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                750       
                                                                 
 dense_1 (Dense)             (None, 10)                160       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 921
Trainable params: 921
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "AUC"],steps_per_execution=None)
# Train the model
fit_output = nn1.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 3s 2ms/step - loss: 0.5984 - accuracy: 0.7023 - auc: 0.7438
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5556 - accuracy: 0.7287 - auc: 0.7817
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5514 - accuracy: 0.7290 - auc: 0.7857
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7305 - auc: 0.7872
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7293 - auc: 0.7873
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.7310 - auc: 0.7886
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7313 - auc: 0.7893
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7317 - auc: 0.7899
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accur

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_AUC = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, AUC: {model_AUC}")

268/268 - 1s - loss: 0.5585 - accuracy: 0.7247 - auc: 0.7811 - 594ms/epoch - 2ms/step
Loss: 0.5585076212882996, Accuracy: 0.7246647477149963, AUC: 0.781107485294342


Autotuning

In [11]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn2 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    units_param = hp.Int('first_units', min_value=1, max_value=10, step=2)
    nn2.add(tf.keras.layers.Dense(units=units_param, activation=activation, input_dim=49))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
      units_param = hp.Int('units_'+ str(i), min_value=1, max_value=10, step=2)
      nn2.add(tf.keras.layers.Dense(units=units_param, activation=activation))

    # Add output layer
    nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn2.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn2

In [ ]:
!pip install keras-tuner

In [13]:
# Define tuner
import keras_tuner as kt
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [14]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 45s]
val_accuracy: 0.713469386100769

Best val_accuracy So Far: 0.728396475315094
Total elapsed time: 00h 20m 43s
INFO:tensorflow:Oracle triggered exit


In [15]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values


{'activation': 'tanh',
 'first_units': 7,
 'num_layers': 2,
 'tuner/bracket': 0,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_0': 9,
 'units_1': 1,
 'units_2': 9,
 'units_3': 7,
 'units_4': 7,
 'units_5': 1}

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=7, activation="sigmoid", input_dim=49))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=9, activation="sigmoid"))
nn2.add(tf.keras.layers.Dense(units=7, activation="sigmoid"))
nn2.add(tf.keras.layers.Dense(units=7, activation="sigmoid"))
nn2.add(tf.keras.layers.Dense(units=9, activation="sigmoid"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 7)                 350       
                                                                 
 dense_9 (Dense)             (None, 9)                 72        
                                                                 
 dense_10 (Dense)            (None, 7)                 70        
                                                                 
 dense_11 (Dense)            (None, 7)                 56        
                                                                 
 dense_12 (Dense)            (None, 9)                 72        
                                                                 
 dense_13 (Dense)            (None, 1)                 10        
                                                                 
Total params: 630
Trainable params: 630
Non-trainable 

In [17]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "AUC"],steps_per_execution=None)
# Train the model
fit_output = nn2.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 3s 3ms/step - loss: 0.6783 - accuracy: 0.5719 - auc: 0.6324
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5817 - accuracy: 0.7295 - auc: 0.7431
Epoch 3/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5742 - accuracy: 0.7313 - auc: 0.7545
Epoch 4/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5730 - accuracy: 0.7315 - auc: 0.7560
Epoch 5/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5725 - accuracy: 0.7316 - auc: 0.7557
Epoch 6/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5723 - accuracy: 0.7309 - auc: 0.7579
Epoch 7/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5720 - accuracy: 0.7317 - auc: 0.7569
Epoch 8/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5717 - accuracy: 0.7316 - auc: 0.7586
Epoch 9/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5714 - accur

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_AUC = nn2a.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, AUC: {model_AUC}")

268/268 - 1s - loss: 0.5739 - accuracy: 0.7259 - auc: 0.7618 - 608ms/epoch - 2ms/step
Loss: 0.5739039182662964, Accuracy: 0.7259474992752075, AUC: 0.761808454990387


 Adjust input data

In [19]:
# Remove columns ORGANIZATION, STATUS
# Change Other Classification to all values less than 500
classifications_to_replace = val_counts.loc[val_counts < 500].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Remove columns ORGANIZATION, STATUS
application_df_2 = application_df.drop(['ORGANIZATION', 'STATUS'], axis=1)



In [20]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies_2 = pd.get_dummies(application_df_2)

# Split our preprocessed data into our features and target arrays
target = app_dummies_2['IS_SUCCESSFUL']
app_dummies_2 = app_dummies_2.drop('IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
(X_train_2, X_test_2, y_train_2, y_test_2) = train_test_split(app_dummies_2, target, random_state=76)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler_2 = scaler.fit(X_train_2)

# Scale the data
X_train_scaled_2 = X_scaler_2.transform(X_train_2)
X_test_scaled_2 = X_scaler_2.transform(X_test_2)

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=7, activation="sigmoid", input_dim=39))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=9, activation="sigmoid"))
nn3.add(tf.keras.layers.Dense(units=7, activation="sigmoid"))
nn3.add(tf.keras.layers.Dense(units=7, activation="sigmoid"))
nn3.add(tf.keras.layers.Dense(units=9, activation="sigmoid"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 7)                 280       
                                                                 
 dense_15 (Dense)            (None, 9)                 72        
                                                                 
 dense_16 (Dense)            (None, 7)                 70        
                                                                 
 dense_17 (Dense)            (None, 7)                 56        
                                                                 
 dense_18 (Dense)            (None, 9)                 72        
                                                                 
 dense_19 (Dense)            (None, 1)                 10        
                                                                 
Total params: 560
Trainable params: 560
Non-trainable 

In [22]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "AUC"],steps_per_execution=None)

In [23]:
# Train the model
fit_output = nn3.fit(X_train_scaled_2, y_train_2, epochs=20)

Epoch 1/20
804/804 [==============================] - 3s 2ms/step - loss: 0.6821 - accuracy: 0.5620 - auc: 0.6184
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5874 - accuracy: 0.7262 - auc: 0.7341
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5803 - accuracy: 0.7264 - auc: 0.7458
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5795 - accuracy: 0.7273 - auc: 0.7480
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5790 - accuracy: 0.7271 - auc: 0.7491
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5784 - accuracy: 0.7261 - auc: 0.7504
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5778 - accuracy: 0.7254 - auc: 0.7528
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5771 - accuracy: 0.7276 - auc: 0.7528
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5768 - accur

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_AUC = nn3.evaluate(X_test_scaled_2,y_test_2,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, AUC: {model_AUC}")

268/268 - 1s - loss: 0.5818 - accuracy: 0.7201 - auc: 0.7566 - 588ms/epoch - 2ms/step
Loss: 0.5817957520484924, Accuracy: 0.7201166152954102, AUC: 0.7566041946411133


 Increasing model complexity of new model

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=15, activation="sigmoid", input_dim=49))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=15, activation="sigmoid"))
nn4.add(tf.keras.layers.Dense(units=15, activation="sigmoid"))
nn4.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))
nn4.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 15)                750       
                                                                 
 dense_21 (Dense)            (None, 15)                240       
                                                                 
 dense_22 (Dense)            (None, 15)                240       
                                                                 
 dense_23 (Dense)            (None, 10)                160       
                                                                 
 dense_24 (Dense)            (None, 10)                110       
                                                                 
 dense_25 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,511
Trainable params: 1,511
Non-traina

In [26]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "AUC"],steps_per_execution=None)
# Train the model
fit_output = nn4.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 3s 2ms/step - loss: 0.6374 - accuracy: 0.6457 - auc: 0.6973
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5744 - accuracy: 0.7297 - auc: 0.7538
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5727 - accuracy: 0.7301 - auc: 0.7572
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5721 - accuracy: 0.7310 - auc: 0.7587
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5717 - accuracy: 0.7312 - auc: 0.7591
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5711 - accuracy: 0.7318 - auc: 0.7612
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5708 - accuracy: 0.7320 - auc: 0.7619
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5705 - accuracy: 0.7315 - auc: 0.7617
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5699 - accur

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_AUC = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, AUC: {model_AUC}")

268/268 - 1s - loss: 0.5604 - accuracy: 0.7257 - auc: 0.7784 - 587ms/epoch - 2ms/step
Loss: 0.5604339241981506, Accuracy: 0.7257142663002014, AUC: 0.7783678770065308


In [28]:
# Save models as HDF5
nn1.save('AlphabetSoupCharity_optimization_1.h5')
nn2.save('AlphabetSoupCharity_optimization_2.h5')
nn3.save('AlphabetSoupCharity_optimization_3.h5')
nn4.save('AlphabetSoupCharity_optimization_4.h5')